In [2]:
import sys
#lets you access python's runtime environment
from pathlib import Path
#sys.path is a built in variable in the sys module and contains a list of directories that is seached through when you do an import
#so we are appending the src directory to that
sys.path.append(str(Path().resolve().parent / "src"))

import config
import pandas as pd

In [3]:
df = pd.read_csv(config.DB_LOCATION)

C:\Users\bnpar\AppData\Local\Temp\ipykernel_63920\26049954.py:1: DtypeWarning: Columns (33,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(config.DB_LOCATION)


In [43]:
#reduced dataframe
red_df = df[['Name', 'Sex', 'Division', 'BodyweightKg', 'WeightClassKg','Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg', 'TotalKg', 'Date', 'Sanctioned', 'Equipment']]

In [5]:
def get_mixed_cols(dataframe):
    all_types_count = {} #nested dict w column heading as outer key. 
    for col in dataframe.columns:
        col_types = dataframe[col].map(type)   #applies type(x) to every x in the column so you get series ['string', 'string', 'float'....]
        all_types_count[col] = col_types.value_counts().to_dict()
    mixed_type_cols = []
    for col in all_types_count:
        no_types = len(all_types_count[col].values())
        #we are only printing columns with mixed data types
        if no_types > 1:
            mixed_type_cols.append(col)
    return mixed_type_cols

In [16]:
mixed_cols = get_mixed_cols(red_df)
print(mixed_cols)

['Division', 'WeightClassKg']


In [17]:
#checking if all the floats are because of NaN
for col in mixed_cols:
    mask = red_df[col].map(type) == float
    float_col = red_df.loc[mask, col]
    float_col = float_col.dropna()
    print(float_col)

Series([], Name: Division, dtype: object)
Series([], Name: WeightClassKg, dtype: object)


All float datatypes are because of NaN entries in WeightClassKg or Division. Want to drop any entries where we dont know what the weight class is. Are okay with not knowing the Division i.e. whether the lifter is a Junior or Open lifter for now. ALso chosen to drop any unsanctioned meets. Also want to filter for Raw powerlifting only (specified in 'Equipment' column). Also want to turn dates into datetime objects.

In [56]:
red_df = red_df[red_df['Equipment'] == 'Raw']
red_df['Date'] = pd.to_datetime(red_df['Date'] , format = '%Y-%m-%d')
red_df = red_df.dropna(axis = 'rows', subset = ['WeightClassKg'])
unsanctioned = red_df[red_df['Sanctioned'] == 'No'].index.to_list()
red_df = red_df.drop(labels = unsanctioned, axis = 'index')

In [48]:
red_df['Date'].head(10)

0    2025-08-17
1    2025-08-17
2    2025-08-17
3    2025-08-17
4    2025-08-17
5    2025-08-17
6    2025-08-17
7    2025-08-17
8    2025-08-17
9    2025-08-17
Name: Date, dtype: object

In [63]:
copy = red_df.copy()
copy['Year'] =  copy['Date'].dt.year
copy['Year'].value_counts()

Year
2019    78325
2025    75901
2024    74120
2018    72359
2023    65798
2017    63116
2016    53710
2022    49866
2021    45351
2015    40075
2014    28919
2020    27650
2013    17484
2012    10781
2011     7304
1985     3918
2010     3645
1984     3530
2009     2391
1982     2340
1983     2325
2008     1460
1981     1387
2007      988
2003      940
1980      791
2004      751
2002      707
2005      594
2006      588
1979      572
1986      501
2001      312
1991      252
2000      152
1990      131
1998       99
1999       95
1993       93
1997       80
1994       72
1996       69
1987       65
1992       50
1974       42
1995       34
1988       23
1973        8
1977        6
1975        2
1978        1
1966        1
1971        1
1972        1
1967        1
1970        1
1969        1
1976        1
Name: count, dtype: int64